In [1]:
import numpy as np
import torch.nn as nn
import matplotlib
import random
import torch

from collections import Counter
from itertools import islice
from sklearn.utils import shuffle
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset

from models.Encoder import *
from models.Decoder import *
from models.utils import *
from utils import *

# set fixed random seed to reproduce results
np.random.seed(42)
random.seed(42)

%matplotlib inline

GPU is available
GPU is available


# Experiment 1b

In [2]:
# define experiment
exp='/exp_1'

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds_train, act_vocab, w2i_acts, i2w_acts, acts_train = load_dataset(exp=exp, split='/train')
_, _, _, cmds_test, _, _, _, acts_test = load_dataset(exp=exp, split='/test')

In [5]:
## create input and output language pairs ##

# training
train_cmd_act_pairs = create_pairs(cmds_train, acts_train)
print("Number of train source-target pairs: {}".format(len(train_cmd_act_pairs)))

# testing
test_cmd_act_pairs = create_pairs(cmds_test, acts_test)
print("Number of test source-target pairs: {}".format(len(test_cmd_act_pairs)))

Number of train source-target pairs: 16728
Number of test source-target pairs: 4182


In [6]:
# show random train command-action pair
random_pair = random.choice(train_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['look', 'left', 'thrice', 'after', 'turn', 'opposite', 'left']
Action: ['I_TURN_LEFT', 'I_TURN_LEFT', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK']


In [7]:
# show random test command-action pair
random_pair = random.choice(test_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['run', 'right', 'thrice', 'after', 'jump', 'right']
Action: ['I_TURN_RIGHT', 'I_JUMP', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN']


In [8]:
cmd_act_pair = pairs2idx(random_pair, w2i_cmds, w2i_acts)
print("Command sequence: {}".format(cmd_act_pair[0]))
print("Action sequence: {}".format(cmd_act_pair[1]))

Command sequence: tensor([14,  4,  7, 10, 12,  4,  2])
Action sequence: tensor([1, 4, 6, 4, 7, 4, 7, 4, 7, 2])


In [9]:
## Hyperparameters for training ##

# source language (i.e., commands) vocabulary size |V_source|
in_size = len(w2i_cmds)

# target language (i.e., actions) vocabulary size |V_target|
out_size = len(w2i_acts)

# size of word embeddings
emb_size = 15

# size of hidden units
hidden_size = 20

# number of layers
n_layers = 1

# number of epochs
n_epochs = 1 # 4-6 epochs seems to be sufficient to learn the mapping

In [10]:
## Instantiate models ##

encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers)
decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers)

C:\Users\lukas\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [11]:
# move models to GPU, if GPU is available (for faster computation)
encoder.to(device)
decoder.to(device)

DecoderLSTM(
  (embedding): Embedding(10, 15)
  (lstm): LSTM(15, 20, dropout=0.5)
  (linear): Linear(in_features=20, out_features=10, bias=True)
)

## Training (1a)

In [ ]:
# train_losses, train_accs, encoder, decoder = train(train_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder, n_epochs)

## Testing (1a)

In [ ]:
# test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)

## Train and test over different numbers of distinct source-target pairs (1b)

In [13]:
def sample_distinct_pairs(cmd_act_pairs:list, ratio:float):
    # randomly shuffle the data set prior to picking distinct examples
    np.random.shuffle(cmd_act_pairs)
    n_lang_pairs = len(cmd_act_pairs)
    n_distinct_samples = int(n_distinct_lang_pairs * ratio)        
    return cmd_act_pairs[:n_distinct_samples]

In [14]:
## experiment 1b train and test loop over pre-defined ratios ##

ratios = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
accs_per_ratio = {}
for ratio in ratios:
    train_samples = sample_distinct_pairs(train_cmd_act_pairs, ratio)
    break
    train_losses, train_accs, encoder, decoder = train(train_samples, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder, n_epochs)
    test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)
    accs_per_ratio[ratio] = test_acc

16728
